In [2]:
import pandas as pd

spending_practical_df = pd.read_table('data/grouping_practical.tsv', index_col="unique_id", dtype={"doctor_id":"object"})
spending_practical_df.head(10)

,doctor_id,specialty,medication,nb_beneficiaries,spending
unique_id,,,,,
NX531425,1255626040,FAMILY PRACTICE,METFORMIN HCL,30,135.24
QG879256,1699761833,FAMILY PRACTICE,ALLOPURINOL,30,715.76
FW363228,1538148804,INTERNAL MEDICINE,LOSARTAN POTASSIUM,146,1056.47
WD733417,1730200619,PSYCHIATRY,OLANZAPINE,13,28226.97
XW149832,1023116894,FAMILY PRACTICE,PRAVASTATIN SODIUM,348,8199.48
QT485324,1952359671,FAMILY PRACTICE,HYDROCHLOROTHIAZIDE,57,247.01
NA293426,1841235223,FAMILY PRACTICE,SEVELAMER CARBONATE,11,4869.32
IF945618,1326095662,INTERNAL MEDICINE,FLUTICASONE/SALMETEROL,20,7832.46
PH384257,1821126830,HEMATOLOGY/ONCOLOGY,ZOLPIDEM TARTRATE,14,65.21


In [13]:
def filter_groups(x):
    # Recall the shape has the number of row and the number of colums
    # we want the first element (hence 0)
    return x.shape[0] > 200 and  x["spending"].sum() > 15000

spending_practical_df_filtered = (spending_practical_df
                                    .groupby("specialty")
                                    .filter(filter_groups))
(spending_practical_df_filtered
  .sort_values(by=["specialty", "nb_beneficiaries", "spending"], 
                                  ascending=[True, False, False]).head()
)

,doctor_id,specialty,medication,nb_beneficiaries,spending,spending_pct
unique_id,,,,,,
PQ631556,1487628210,CARDIOLOGY,METOPROLOL SUCCINATE,795,16943.11,0.884394
DL858715,1629098413,CARDIOLOGY,LISINOPRIL,564,2836.26,0.148047
DA550259,1083603401,CARDIOLOGY,LISINOPRIL,496,2452.04,0.127991
EH370508,1750388740,CARDIOLOGY,LOVASTATIN,449,4427.32,0.231097
JB610796,1164433868,CARDIOLOGY,ROSUVASTATIN CALCIUM,442,107540.09,5.613361


In [4]:
def my_function(x):
    return (x   / x.sum() ) * 100

spending_by_specialty =    spending_practical_df.groupby("specialty") 
spending_practical_df["spending_pct"] = (spending_by_specialty['spending']
                                        .transform(my_function))
spending_practical_df.head()

medication_spendng_pct = (spending_practical_df
                            .groupby(["specialty", "medication"])["spending_pct"]
                            .sum().reset_index()
                         )

In [5]:
medication_spendng_pct.head()

,specialty,medication,spending_pct
0,ADDICTION MEDICINE,BUSPIRONE HCL,88.894203
1,ADDICTION MEDICINE,LAMOTRIGINE,8.979849
2,ADDICTION MEDICINE,LORAZEPAM,2.125948
3,ALLERGY/IMMUNOLOGY,ALBUTEROL SULFATE,3.553199
4,ALLERGY/IMMUNOLOGY,AMOXICILLIN/POTASSIUM CLAV,0.196089


In [6]:
# This code may not work in certain cass, can you guess how?
def filter_by_top_2(x):
    return x["spending_pct"].nlargest(2).sum() > 80

medication_spendng_pct.groupby("specialty").filter(filter_by_top_2).head()

,specialty,medication,spending_pct
0,ADDICTION MEDICINE,BUSPIRONE HCL,88.894203
1,ADDICTION MEDICINE,LAMOTRIGINE,8.979849
2,ADDICTION MEDICINE,LORAZEPAM,2.125948
47,CARDIAC ELECTROPHYSIOLOGY,ATENOLOL,0.288760
48,CARDIAC ELECTROPHYSIOLOGY,DIGOXIN,2.390260


In [7]:
def get_2_largest(x):
    
    return x.nlargest(2, "spending_pct")

(medication_spendng_pct.groupby("specialty")
 .filter(filter_by_top_2)
 .groupby("specialty")
 .apply(get_2_largest).head(10))

specialty  \
specialty                                                  
ADDICTION MEDICINE        0           ADDICTION MEDICINE   
                          1           ADDICTION MEDICINE   
CARDIAC ELECTROPHYSIOLOGY 56   CARDIAC ELECTROPHYSIOLOGY   
                          50   CARDIAC ELECTROPHYSIOLOGY   
CARDIAC SURGERY           61             CARDIAC SURGERY   
                          60             CARDIAC SURGERY   
CERTIFIED NURSE MIDWIFE   194    CERTIFIED NURSE MIDWIFE   
                          193    CERTIFIED NURSE MIDWIFE   
CLINIC/CENTER             195              CLINIC/CENTER   
                          196              CLINIC/CENTER   

                                                   medication  spending_pct  
specialty                                                                    
ADDICTION MEDICINE        0                     BUSPIRONE HCL     88.894203  
                          1                       LAMOTRIGINE      8.979849  
CARDIAC ELECTROPHYSIOLOGY 56                      RIVAROXABAN     75.017303  
                          50                  DRONEDARONE HCL     11.050018  
CARDIAC SURGERY           61   INSULIN GLARGINE,HUM.REC.ANLOG     96.437603  
                          60        HYDROCODONE/ACETAMINOPHEN      3.562397  
CERTIFIED NURSE MIDWIFE   194                      MIRABEGRON     93.493400  
                          193     MEDROXYPROGESTERONE ACETATE      6.506600  
CLINIC/CENTER             195          ESOMEPRAZOLE MAGNESIUM     93.414487  
                          196                     MIRTAZAPINE      6.585513